In this notebook I define the functions that describe the dynamical evolution of the densities of agents having each possible vocabulary and integrate the equations numerically. Three cases are considered according to the different agreement rule: pairwise, higher-order (intersection) and higher-order (union). Notice that the higher-order versions consider up to 2-simplices. 

In [1]:
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import palettable as pltt
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
%matplotlib inline

plt.rcParams['xtick.major.width'] = 1.2
plt.rcParams['ytick.major.width'] = 1.2
plt.rcParams['xtick.minor.size'] = 2
plt.rcParams['xtick.minor.width'] = 1.2
plt.rcParams['ytick.minor.size'] = 2
plt.rcParams['ytick.minor.width'] = 1.2
plt.rcParams['axes.linewidth'] = 1.2

## Functions

In [2]:
#I start with the pairwise version
def pwNGCM_dn(t, n, beta, p): 
        
    dn_a = -n[0]*n[1]+((3/2)*beta-1/2)*n[0]*(1-n[0]-n[1]-p)+beta*(1-n[0]-n[1]-p)**2+(3/2)*beta*p*(1-n[0]-n[1]-p)
    dn_b = -n[0]*n[1]+((3/2)*beta-1/2)*n[1]*(1-n[0]-n[1]-p)+beta*(1-n[0]-n[1]-p)**2-p*n[1]
    dn = [dn_a, dn_b]
    return dn

#Higher-order 'intersection' version including 2-simplices (triangles)
def NGCM_intersection_dn(t, n, beta, p): 
        
    dn_a = -2*n[0]**2*n[1]+((5*beta-2)/2)*n[0]**2*(1-n[0]-n[1]-p)-2*n[0]*n[1]**2-3*n[0]*n[1]*(1-n[0]-n[1]-p)+(4*beta-1)*n[0]*(1-n[0]-n[1]-p)**2+((3/2)*beta)*(1-n[0]-n[1]-p)**3+(5*beta/2)*p**2*(1-n[0]-n[1]-p)-2*p*n[0]*n[1]+(5*beta-1)*p*n[0]*(1-n[0]-n[1]-p)+4*beta*p*(1-n[0]-n[1]-p)**2
    dn_b = -2*n[1]**2*n[0]+((5*beta-2)/2)*n[1]**2*(1-n[0]-n[1]-p)-2*n[1]*n[0]**2-3*n[1]*n[0]*(1-n[0]-n[1]-p)+(4*beta-1)*n[1]*(1-n[0]-n[1]-p)**2+((3/2)*beta)*(1-n[0]-n[1]-p)**3-2*p**2*n[1]-4*p*n[0]*n[1]-2*p*n[1]**2-3*p*n[1]*(1-n[0]-n[1]-p)
    dn = [dn_a, dn_b]
    return dn

#Higher-order 'union' version including 2-simplices (triangles)
def NGCM_union_dn(t, n, beta, p): 

    dn_a = 2*(beta-1)*n[0]**2*n[1]+\
    ((5/2)*beta-1)*n[0]**2*(1-n[0]-n[1]-p)-\
    2*n[0]*n[1]**2+\
    (6*beta-3)*n[0]*n[1]*(1-n[0]-n[1]-p)+\
    (4*beta-1)*n[0]*(1-n[0]-n[1]-p)**2+\
    3*beta*n[1]*(1-n[0]-n[1]-p)**2+\
    (3/2)*beta*(1-n[0]-n[1]-p)**3+\
    2*beta*p**2*n[1]+\
    (5/2)*beta*p**2*(1-n[0]-n[1]-p)+\
    2*(2*beta-1)*p*n[0]*n[1]+\
    (5*beta-1)*p*n[0]*(1-n[0]-n[1]-p)+\
    6*beta*p*n[1]*(1-n[0]-n[1]-p)+\
    4*beta*p*(1-n[0]-n[1]-p)**2

    dn_b = 2*(beta-1)*n[1]**2*n[0]+\
    ((5/2)*beta-1)*n[1]**2*(1-n[0]-n[1]-p)-\
    2*n[1]*n[0]**2+\
    (6*beta-3)*n[1]*n[0]*(1-n[0]-n[1]-p)+\
    (4*beta-1)*n[1]*(1-n[0]-n[1]-p)**2+\
    3*beta*n[0]*(1-n[0]-n[1]-p)**2+\
    (3/2)*beta*(1-n[0]-n[1]-p)**3-\
    2*p**2*n[1]-\
    2*p*n[1]**2+\
    (3*beta-3)*p*n[1]*(1-n[0]-n[1]-p)+\
    2*beta*p*(1-n[0]-n[1]-p)**2-\
    4*p*n[0]*n[1]
    
    dn = [dn_a, dn_b]
    return dn

def integrate_curves(fun, beta, p, t_max, n0, t_min=0., dense_output=False, t_eval=None):
    if t_eval=='auto':
        t_eval = np.linspace(t_min, t_max)
    res = solve_ivp(fun=fun, t_span=(t_min, t_max), y0=n0,
                    t_eval=t_eval, args=[beta, p], dense_output=dense_output)
    t = res.t
    ns = res.y.T
    return t, ns    

# Numerical integration

## Stationary values as a function of $p$

Pairwise, running and saving

In [18]:
betas = [0.33,0.4,0.7]#,1]
ps = np.linspace(0.0, 0.1, 100)
rule='pairwise'

t_max = 2000
t_eval = None#range(t_max)

for beta in betas:
    
    outfname = '../Results/MF/%s/fixed_beta%.4f_varp.csv'%(rule,beta)
    f = open(outfname,'w')
    f.write('p,n_Ap,n_B\n')
    
    ns_longtime_pairwise = []
    
    for p in ps:
        n_a0 = 0.
        n_b0 = 1-n_a0-p
        n0 = [n_a0, n_b0]

        x, ns = integrate_curves(pwNGCM_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a and appending
        ns_longtime_pairwise.append(np.array([ns[-1][0]+p, ns[-1][1]])) 

    for (nap, nb), p in zip(ns_longtime_pairwise, ps):
        line = str(p)+','+str(nap)+','+str(nb)+'\n'
        f.write(line)
    f.close()

Higher-order (intersection), running and saving

In [19]:
betas = [0.33,0.4,0.7]#,1]
ps = np.linspace(0.0, 0.1, 100)
rule='intersection'

t_max = 2000
t_eval = None#range(t_max)

for beta in betas:
    
    outfname = '../Results/MF/%s/fixed_beta%.4f_varp.csv'%(rule,beta)
    f = open(outfname,'w')
    f.write('p,n_Ap,n_B\n')
    
    ns_longtime_intersection = []
    
    for p in ps:
        n_a0 = 0.
        n_b0 = 1-n_a0-p
        n0 = [n_a0, n_b0]

        x, ns = integrate_curves(NGCM_intersection_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a and appending
        ns_longtime_intersection.append(np.array([ns[-1][0]+p, ns[-1][1]])) 

    for (nap, nb), p in zip(ns_longtime_intersection, ps):
        line = str(p)+','+str(nap)+','+str(nb)+'\n'
        f.write(line)
    f.close()

Higher-order (union), running and saving

In [20]:
betas = [0.33,0.4,0.7]#,1]
ps = np.linspace(0.0, 0.1, 100)
rule='union'

t_max = 2000
t_eval = None#range(t_max)

for beta in betas:
    
    outfname = '../Results/MF/%s/fixed_beta%.4f_varp.csv'%(rule,beta)
    f = open(outfname,'w')
    f.write('p,n_Ap,n_B\n')
    
    ns_longtime_union = []
    
    for p in ps:
        n_a0 = 0.
        n_b0 = 1-n_a0-p
        n0 = [n_a0, n_b0]

        x, ns = integrate_curves(NGCM_union_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a and appending
        ns_longtime_union.append(np.array([ns[-1][0]+p, ns[-1][1]])) 

    for (nap, nb), p in zip(ns_longtime_union, ps):
        line = str(p)+','+str(nap)+','+str(nb)+'\n'
        f.write(line)
    f.close()

## Stationary values as a function of $\beta$

Pairwise, running and saving

In [19]:
betas = np.linspace(0, 1, 100)
ps = [0.04]#[0.02, 0.08, 0.16]
rule='pairwise'

t_max = 2000
t_eval = None#range(t_max)

for p in ps:
    
    outfname = '../Results/MF/%s/fixed_p%.4f_varbeta.csv'%(rule,p)
    f = open(outfname,'w')
    f.write('beta,n_Ap,n_B\n')
    
    ns_longtime_pairwise = []
    
    n_a0 = 0.
    n_b0 = 1-n_a0-p
    n0 = [n_a0, n_b0]

    for beta in betas:

        x, ns = integrate_curves(pwNGCM_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a and appending
        ns_longtime_pairwise.append(np.array([ns[-1][0]+p, ns[-1][1]])) 

    for (nap, nb), beta in zip(ns_longtime_pairwise, betas):
        line = str(beta)+','+str(nap)+','+str(nb)+'\n'
        f.write(line)
    f.close()

Higher-order (intersection), running and saving

In [17]:
betas = np.linspace(0, 1, 100)
ps = [0.04]#[0.02, 0.08, 0.16]
rule='intersection'

t_max = 2000
t_eval = None#range(t_max)

for p in ps:
    
    outfname = '../Results/MF/%s/fixed_p%.4f_varbeta.csv'%(rule,p)
    f = open(outfname,'w')
    f.write('beta,n_Ap,n_B\n')
    
    ns_longtime_intersection = []
    
    n_a0 = 0.
    n_b0 = 1-n_a0-p
    n0 = [n_a0, n_b0]

    for beta in betas:

        x, ns = integrate_curves(NGCM_intersection_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a and appending
        ns_longtime_intersection.append(np.array([ns[-1][0]+p, ns[-1][1]])) 

    for (nap, nb), beta in zip(ns_longtime_intersection, betas):
        line = str(beta)+','+str(nap)+','+str(nb)+'\n'
        f.write(line)
    f.close()

Higher-order (union), running and saving

In [18]:
betas = np.linspace(0, 1, 100)
ps = [0.04]#[0.02, 0.08, 0.16]
rule='union'

t_max = 2000
t_eval = None#range(t_max)

for p in ps:
    
    outfname = '../Results/MF/%s/fixed_p%.4f_varbeta.csv'%(rule,p)
    f = open(outfname,'w')
    f.write('beta,n_Ap,n_B\n')
    
    ns_longtime_union = []
    
    n_a0 = 0.
    n_b0 = 1-n_a0-p
    n0 = [n_a0, n_b0]

    for beta in betas:
        
        x, ns = integrate_curves(NGCM_union_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a and appending
        ns_longtime_union.append(np.array([ns[-1][0]+p, ns[-1][1]])) 

    for (nap, nb), beta in zip(ns_longtime_union, betas):
        line = str(beta)+','+str(nap)+','+str(nb)+'\n'
        f.write(line)
    f.close()

## Computing heatmaps $\beta$ vs $p$ (SLOW)

In [ ]:
betas = np.linspace(0.001,1,100)
ps = np.linspace(0.001,0.15,100)
t_max = 2000
t_eval = None

out_fname = "../Results/NGCM_intersection_heatmap_beta_p.csv"
heatmap = open(out_fname, 'w')
header = "beta,p,n_a,n_b\n"
heatmap.write(header)

for p in ps:
    print('p=%.3f...'%p)
    n_a0 = 0.
    n_b0 = 1-n_a0-p
    n0 = [n_a0, n_b0]

    for beta in betas:
        x, ns = integrate_curves(NGCM_intersection_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a
        n_a_longt = ns[-1][0]+p
        n_b_longt = ns[-1][1]
        
        line = str(beta)+','+str(p)+','+str(n_a_longt)+','+str(n_b_longt)+'\n'
        heatmap.write(line)
   
heatmap.close()

In [ ]:
betas = np.linspace(0.001,1,100)
ps = np.linspace(0.001,0.15,100)
t_max = 2000
t_eval = None

out_fname = "../Results/NGCM_union_heatmap_beta_p.csv"
heatmap = open(out_fname, 'w')
header = "beta,p,n_a,n_b\n"
heatmap.write(header)

for p in ps:
    print('p=%.3f...'%p)
    n_a0 = 0.
    n_b0 = 1-n_a0-p
    n0 = [n_a0, n_b0]

    for beta in betas:
        x, ns = integrate_curves(NGCM_union_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a
        n_a_longt = ns[-1][0]+p
        n_b_longt = ns[-1][1]
        
        line = str(beta)+','+str(p)+','+str(n_a_longt)+','+str(n_b_longt)+'\n'
        heatmap.write(line)
   
heatmap.close()

In [ ]:
betas = np.linspace(0.001,1,100)
ps = np.linspace(0.001,0.15,100)
t_max = 2000
t_eval = None

out_fname = "../Results/NGCM_pairwise_heatmap_beta_p.csv"
heatmap = open(out_fname, 'w')
header = "beta,p,n_a,n_b\n"
heatmap.write(header)

for p in ps:
    print('p=%.3f...'%p)
    n_a0 = 0.
    n_b0 = 1-n_a0-p
    n0 = [n_a0, n_b0]

    for beta in betas:
        x, ns = integrate_curves(pwNGCM_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a
        n_a_longt = ns[-1][0]+p
        n_b_longt = ns[-1][1]
        
        line = str(beta)+','+str(p)+','+str(n_a_longt)+','+str(n_b_longt)+'\n'
        heatmap.write(line)
   
heatmap.close()

## Denser heatmaps for 3d plots (SLOW)

In [3]:
betas = np.linspace(0.001,1,150)
ps = np.linspace(0.001,0.15,200)
t_max = 2000
t_eval = None

out_fname = "../Results/NGCM_intersection_dense_heatmap_beta_p.csv"
heatmap = open(out_fname, 'w')
header = "beta,p,n_a,n_b\n"
heatmap.write(header)

for p in ps:
    print('p=%.3f...'%p)
    n_a0 = 0.
    n_b0 = 1-n_a0-p
    n0 = [n_a0, n_b0]

    for beta in betas:
        x, ns = integrate_curves(NGCM_intersection_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a
        n_a_longt = ns[-1][0]+p
        n_b_longt = ns[-1][1]
        
        line = str(beta)+','+str(p)+','+str(n_a_longt)+','+str(n_b_longt)+'\n'
        heatmap.write(line)
   
heatmap.close()

p=0.001...
p=0.002...
p=0.002...
p=0.003...
p=0.004...
p=0.005...
p=0.005...
p=0.006...
p=0.007...
p=0.008...
p=0.008...
p=0.009...
p=0.010...
p=0.011...
p=0.011...
p=0.012...
p=0.013...
p=0.014...
p=0.014...
p=0.015...
p=0.016...
p=0.017...
p=0.017...
p=0.018...
p=0.019...
p=0.020...
p=0.020...
p=0.021...
p=0.022...
p=0.023...
p=0.023...
p=0.024...
p=0.025...
p=0.026...
p=0.026...
p=0.027...
p=0.028...
p=0.029...
p=0.029...
p=0.030...
p=0.031...
p=0.032...
p=0.032...
p=0.033...
p=0.034...
p=0.035...
p=0.035...
p=0.036...
p=0.037...
p=0.038...
p=0.038...
p=0.039...
p=0.040...
p=0.041...
p=0.041...
p=0.042...
p=0.043...
p=0.044...
p=0.044...
p=0.045...
p=0.046...
p=0.047...
p=0.047...
p=0.048...
p=0.049...
p=0.050...
p=0.050...
p=0.051...
p=0.052...
p=0.053...
p=0.053...
p=0.054...
p=0.055...
p=0.056...
p=0.056...
p=0.057...
p=0.058...
p=0.059...
p=0.059...
p=0.060...
p=0.061...
p=0.062...
p=0.062...
p=0.063...
p=0.064...
p=0.065...
p=0.065...
p=0.066...
p=0.067...
p=0.068...
p=0.068...

In [4]:
betas = np.linspace(0.001,1,150)
ps = np.linspace(0.001,0.15,200)
t_max = 2000
t_eval = None

out_fname = "../Results/NGCM_union_dense_heatmap_beta_p.csv"
heatmap = open(out_fname, 'w')
header = "beta,p,n_a,n_b\n"
heatmap.write(header)

for p in ps:
    print('p=%.3f...'%p)
    n_a0 = 0.
    n_b0 = 1-n_a0-p
    n0 = [n_a0, n_b0]

    for beta in betas:
        x, ns = integrate_curves(NGCM_union_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a
        n_a_longt = ns[-1][0]+p
        n_b_longt = ns[-1][1]
        
        line = str(beta)+','+str(p)+','+str(n_a_longt)+','+str(n_b_longt)+'\n'
        heatmap.write(line)
   
heatmap.close()

p=0.001...
p=0.002...
p=0.002...
p=0.003...
p=0.004...
p=0.005...
p=0.005...
p=0.006...
p=0.007...
p=0.008...
p=0.008...
p=0.009...
p=0.010...
p=0.011...
p=0.011...
p=0.012...
p=0.013...
p=0.014...
p=0.014...
p=0.015...
p=0.016...
p=0.017...
p=0.017...
p=0.018...
p=0.019...
p=0.020...
p=0.020...
p=0.021...
p=0.022...
p=0.023...
p=0.023...
p=0.024...
p=0.025...
p=0.026...
p=0.026...
p=0.027...
p=0.028...
p=0.029...
p=0.029...
p=0.030...
p=0.031...
p=0.032...
p=0.032...
p=0.033...
p=0.034...
p=0.035...
p=0.035...
p=0.036...
p=0.037...
p=0.038...
p=0.038...
p=0.039...
p=0.040...
p=0.041...
p=0.041...
p=0.042...
p=0.043...
p=0.044...
p=0.044...
p=0.045...
p=0.046...
p=0.047...
p=0.047...
p=0.048...
p=0.049...
p=0.050...
p=0.050...
p=0.051...
p=0.052...
p=0.053...
p=0.053...
p=0.054...
p=0.055...
p=0.056...
p=0.056...
p=0.057...
p=0.058...
p=0.059...
p=0.059...
p=0.060...
p=0.061...
p=0.062...
p=0.062...
p=0.063...
p=0.064...
p=0.065...
p=0.065...
p=0.066...
p=0.067...
p=0.068...
p=0.068...

In [5]:
betas = np.linspace(0.001,1,150)
ps = np.linspace(0.001,0.15,200)
t_max = 2000
t_eval = None

out_fname = "../Results/NGCM_pairwise_dense_heatmap_beta_p.csv"
heatmap = open(out_fname, 'w')
header = "beta,p,n_a,n_b\n"
heatmap.write(header)

for p in ps:
    print('p=%.3f...'%p)
    n_a0 = 0.
    n_b0 = 1-n_a0-p
    n0 = [n_a0, n_b0]

    for beta in betas:
        x, ns = integrate_curves(pwNGCM_dn, beta, p, t_max, n0, t_eval=t_eval)
        #adding back the minorities to n_a
        n_a_longt = ns[-1][0]+p
        n_b_longt = ns[-1][1]
        
        line = str(beta)+','+str(p)+','+str(n_a_longt)+','+str(n_b_longt)+'\n'
        heatmap.write(line)
   
heatmap.close()

p=0.001...
p=0.002...
p=0.002...
p=0.003...
p=0.004...
p=0.005...
p=0.005...
p=0.006...
p=0.007...
p=0.008...
p=0.008...
p=0.009...
p=0.010...
p=0.011...
p=0.011...
p=0.012...
p=0.013...
p=0.014...
p=0.014...
p=0.015...
p=0.016...
p=0.017...
p=0.017...
p=0.018...
p=0.019...
p=0.020...
p=0.020...
p=0.021...
p=0.022...
p=0.023...
p=0.023...
p=0.024...
p=0.025...
p=0.026...
p=0.026...
p=0.027...
p=0.028...
p=0.029...
p=0.029...
p=0.030...
p=0.031...
p=0.032...
p=0.032...
p=0.033...
p=0.034...
p=0.035...
p=0.035...
p=0.036...
p=0.037...
p=0.038...
p=0.038...
p=0.039...
p=0.040...
p=0.041...
p=0.041...
p=0.042...
p=0.043...
p=0.044...
p=0.044...
p=0.045...
p=0.046...
p=0.047...
p=0.047...
p=0.048...
p=0.049...
p=0.050...
p=0.050...
p=0.051...
p=0.052...
p=0.053...
p=0.053...
p=0.054...
p=0.055...
p=0.056...
p=0.056...
p=0.057...
p=0.058...
p=0.059...
p=0.059...
p=0.060...
p=0.061...
p=0.062...
p=0.062...
p=0.063...
p=0.064...
p=0.065...
p=0.065...
p=0.066...
p=0.067...
p=0.068...
p=0.068...